Con EvomSA y LinearSVC
4 Modelos combinados 12 (de los modelos no tiene al MeOffendEs pre-entrenado)
y 7 DS en Es incluyendo MeOfendeEs (como train/text)
12x7=84

Falta:
Vulgarity

Solo Aplico EvoMSA. Faltarian los otros (Idf-Tf, NB, mTC corre con LinearSVC, EvoMSA corre con EvoDAGE + GaussianNB, FastText + Sinonimos(transitivos) + NCR-Emotion-Lexicon-Es(MX.bin + Cluster) )
++ Hate Speech Detection Using a Convolution-LSTM Based Deep Neural Network
++ Detection of Offensive Tweets: A Comparative Study
++ Varios articulos en: C:\Users\juan\Documents\DCCD\articulos
k-fold
Intervalo de Confianza
Jaccard similarity From https://text-models.readthedocs.io/en/latest/voc.html#voc para establecer Relacion
cosine distanc
PCA


Genetic Algorithm + EvoDAG
EvoMSA

En el PDF "las17 CD Grapgh.pdf" es un articulo acerda de la CD graph.

demsar06a.pdf "Statistical Comparisons of Classifiers over Multiple Data Sets" tiene graficas comparativas de clasificadores.

http://localhost:8889/lab

# Ya esta en la tabla EvoMSA_dsEs_mAll_LienarSVC

Survey of Text Classifiers for Offensive Language in Short Text used in RRSS


In [1]:
from glob import glob
import winsound
import time
import pandas as pd
import sklearn.model_selection as model_selection
from EvoMSA.base import EvoMSA
from EvoMSA.model import TextModelInv
from microtc.textmodel import TextModel
from sklearn import metrics
from microtc.utils import load_model
from microtc.utils import save_model
from EvoMSA.utils import download

usar = "sklearn.svm.LinearSVC"
# usar = "sklearn.naive_bayes.BernoulliNB"
# usar = "sklearn.naive_bayes.MultinomialNB"

In [2]:
#misoginia = download('misoginia_Es.evomsa')
#save_model(misoginia, 'models/misoginia_Es.evomsa')

In [3]:
reverse_model = [load_model('models\\text_model_inv_simple.evomsa'), usar]
haha_model = [load_model('models\\haha2018_Es.evomsa'), usar]
mexa3t_model = [load_model('models\\mexa3t2018_aggress_Es.evomsa'), usar]
misoginia_model = [load_model('models\\misoginia_Es.evomsa'), usar]
#misoginia_model = [download('misoginia_Es.evomsa'), usar]

C:\Users\juan\anaconda3\envs\orange3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator LinearSVC from version 0.22.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\juan\anaconda3\envs\orange3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator GaussianNB from version 0.22.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:
M = [
      [],
      [reverse_model], 
      [haha_model], 
      [mexa3t_model], 
      [misoginia_model], 

      [reverse_model, haha_model], 
      [reverse_model, misoginia_model], 
      [reverse_model, mexa3t_model], 
      [haha_model, mexa3t_model], 
      [haha_model, misoginia_model], 
    
      [haha_model, mexa3t_model, misoginia_model], 
    
      [reverse_model, haha_model, mexa3t_model, misoginia_model]
    ]

M = [[misoginia_model, reverse_model, haha_model, mexa3t_model]]

In [5]:
from EvoMSA import __version__ as version
print(version)

1.4.0


In [6]:
'''
Para TextosCortos, no es recomendable quitar los Stopwords

from nltk.corpus import stopwords
from textblob import TextBlob

def removeStopwords( texto):
    blob = TextBlob(texto.lower()).words
    outputlist = [word for word in blob if word not in stopwords.words('spanish')]
    return(' '.join(word for word in outputlist))

print(removeStopwords('las Cosas sin sentidos DCHAS ichas por el guason el año pasado han sido una nefasta decisión de todos y no o si por las dudas de los angeles mal escirto'))
'''

"\nPara TextosCortos, no es recomendable quitar los Stopwords\n\nfrom nltk.corpus import stopwords\nfrom textblob import TextBlob\n\ndef removeStopwords( texto):\n    blob = TextBlob(texto.lower()).words\n    outputlist = [word for word in blob if word not in stopwords.words('spanish')]\n    return(' '.join(word for word in outputlist))\n\nprint(removeStopwords('las Cosas sin sentidos DCHAS ichas por el guason el año pasado han sido una nefasta decisión de todos y no o si por las dudas de los angeles mal escirto'))\n"

In [ ]:
start = time.time()

## se llama semeval2018_anger_Es

# los models son es Es, entonces solo ds en Es
# fnames = glob("dataset_Es/*_train.json")
fnames = glob("dataset/haha2018_Es_train.json")
fnames.sort()

p_ds_m = []
for fn in fnames:
    
    train_df = pd.read_json(fn, lines=True)
    test_df = pd.read_json(fn.replace("_train", "_test"), lines=True)
    
    # recortarlos
    #train_df = train_df.loc[0:49, ['text', 'klass']]
    #test_df = test_df.loc[0:49, ['text', 'klass']]

    if False: # BiFrases ya no aplican
        _ = train_df.iloc[0]
        bi_train_df = pd.DataFrame([_], columns = ['text', 'klass'])
        for index, row in train_df.iloc[1:].iterrows():
            try:
                row['text'] = removeStopwords(row['text'])
                bi_train_df =  bi_train_df.append(row)
                if _['klass'] == row['klass']:
                    bi_frase = {'text': _['text'] + ' ' + row['text'], 'klass':row['klass']}
                    bi_train_df = bi_train_df.append(bi_frase, ignore_index=True)
                    _ =  {'text': '_V_A_C_I_A_', 'klass': None}
                else:   
                    _ = row
            except Exception as exc:
                print(exc)
                _ = row
                continue
        train_df = bi_train_df
    
    X_train = train_df[['text']]
    y_train = train_df[['klass']]
    
    X_test = test_df[['text']]
    y_test = test_df[['klass']]
    
    ds_n = fn.replace("dataset_Es/",'').replace("dataset_Es\\",'').replace('_train.json','')
    #p_m = []    
    print('__',ds_n,'___')
    for m in M:
        print('> ', m)
        evo = EvoMSA(TR=True, B4MSA=False, lang='es', Emo=True, Aggress=True, stacked_method='sklearn.svm.LinearSVC',models=m) 
        # sklearn.svm.LinearSVC
        # sklearn.naive_bayes.GaussianNB
        evo.fit(X_train.squeeze(), y_train.squeeze())
    
        pred = evo.predict(X_test.squeeze())
        
        recall_score = metrics.recall_score(pred, y_test, average="macro")
        precision_score = metrics.precision_score(pred, y_test, average="macro")
        f1_score = metrics.f1_score(pred, y_test, average="macro")
        f1_score_micro = metrics.recall_score(pred, y_test, average="micro")
    
        _ = [ds_n, recall_score, f1_score, precision_score, f1_score_micro]

        p_ds_m.append(_)
        print("{0}, {1:1.6f}, {2:1.6f}, {3:1.6f}, {4:1.6f}".format(_[0], _[1], _[2], _[3], _[4]))
        
        winsound.Beep(4500, 300)
    #print(p)
    #p_ds.append(p_m)
    winsound.Beep(3000, 900)
    
    #break
    
winsound.Beep(2500, 1600)    

for p in p_ds_m:
    # print(p)
    print("{0}, {1:1.6f}, {2:1.6f}, {3:1.6f}, {4:1.6f}".format(p[0], p[1], p[2], p[3], p[4]))    
    # print("%.4f, %.4f, %.4f, %.4f, %.4f" % (accuracy, macro_f1, macro_recall, macro_precision, micro_f1))

print(time.time() - start, "seconds")

In [20]:
print('Terminado')

Terminado


In [21]:
for p in p_ds_m:
    # print(p)
    print("{0:1.6f}, {1:1.6f}, {2:1.6f}, {3:1.6f}".format(p[1], p[2], p[3], p[4]))    


0.847769, 0.844047, 0.840919, 0.855500


In [ ]:
save_model(p_ds, 'EvoMSA_dsEs_mAll_LinearSVC.scores')    

In [ ]:
print('ds_n, recall_score, f1_score, precision_score, f1_score_micro')